# Time Series Forecasting with SoaM

In this notebook we will show how to make predictions and persist them using SoaM Forcaster.

In [1]:
import io

from darts import models
import matplotlib.pyplot as plt
from muttlib.dbconn import PgClient
import pandas as pd
import requests

from soam.cfg import get_db_cred
from soam.forecaster import Forecaster
from soam.runner import PipelineRun, StepRun
from soam.savers import CSVSaver, DBSaver

In [2]:
url = "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_retail_sales.csv"
df = pd.read_csv(url)
df.tail()

,ds,y
288,2016-01-01,400928
289,2016-02-01,413554
290,2016-03-01,460093
291,2016-04-01,450935
292,2016-05-01,471421


We plot the sample data:

In [3]:
my_model = models.Prophet(weekly_seasonality=False, daily_seasonality=False)

In [4]:
forecaster = Forecaster(my_model)

In [5]:
predictions = forecaster.run(raw_series=df, output_length=7)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [6]:
db_cred = get_db_cred("../../soam/settings.ini")
db = PgClient(**db_cred)

In [8]:
saver = DBSaver(db)
pipe = PipelineRun()
run_id = saver.save_pipeline_run(pipe)
step_run = StepRun(forecaster, run_id)
step_run_id = saver.save_step_run(step_run)
saver.save_forecaster(forecaster, step_run_id)